In [ ]:
# Third-party
from astropy.io import ascii
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline

In [ ]:
v_sun = [-11.1, -234., 7.25]*u.km/u.s

gc_frame = coord.Galactocentric(z_sun=0*u.pc)

In [ ]:
a13 = ascii.read("../data/A13.txt")
gass = ascii.read("../data/Crane2003-StarInfo.txt")
triand = ascii.read("../data/TriAnd.txt")

triand = triand[(triand['group'] == 'TriAnd1') | (triand['group'] == 'TriAnd2')]
a13 = a13[a13['member'] == 'Y']

In [ ]:
def v_gsr(c):
    c = c.transform_to(coord.Galactic)
    cart_data = c.data.represent_as(coord.CartesianRepresentation)
    v_proj = coord.CartesianRepresentation(v_sun).dot(cart_data / cart_data.norm())
    return c.radial_velocity - v_proj

In [ ]:
print(a13.colnames)
print(gass.colnames)
print(triand.colnames)

In [ ]:
a13_c = coord.Galactic(l=a13['l']*u.deg, b=a13['b']*u.deg,
                        distance=a13['distance']*u.kpc,
                        pm_l_cosb=0*u.mas/u.yr, pm_b=0*u.mas/u.yr,
                        radial_velocity=a13['vhel']*u.km/u.s)
a13_verr = a13['verr']

gass_c = coord.Galactic(l=gass['l']*u.deg, b=gass['b']*u.deg,
                        distance=gass['d']*u.kpc,
                        pm_l_cosb=0*u.mas/u.yr, pm_b=0*u.mas/u.yr,
                        radial_velocity=gass['v_hel']*u.km/u.s)
gass_verr = gass['v_err']

triand_c = coord.Galactic(l=triand['l']*u.deg, b=triand['b']*u.deg,
                          distance=triand['distance']*u.kpc,
                          pm_l_cosb=0*u.mas/u.yr, pm_b=0*u.mas/u.yr,
                          radial_velocity=triand['vhel']*u.km/u.s)
triand_verr = triand['verr']

## TriAnd 1 vs. 2

In [ ]:
triand1_c = triand_c[triand['group'] == 'TriAnd1']
triand2_c = triand_c[triand['group'] == 'TriAnd2']

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6, 5))

for i in range(n_lines):
    idx = (gal_sim.l[:,i] > 80*u.deg) & (gal_sim.l[:,i] < 275*u.deg)
    v = v_gsr(gal_sim[idx,i]).value
    ax.plot(gal_sim.l[idx,i].degree, v, lw=lws[i], **line_style)
    ax.text(270, v[-1]+label_offset[i], 'R={0:.0f} kpc'.format(cyl.rho[0,i].value), 
            rotation=10, **label_style)

ax.errorbar(gass_c.l.degree, v_gsr(gass_c).value, label='GASS',
            marker='o', color='#1b9e77', **data_style)

ax.errorbar(a13_c.l.degree, v_gsr(a13_c).value, label='A13',
            marker='s', color='#d95f02', **data_style)

ax.errorbar(triand1_c.l.degree, v_gsr(triand1_c).value, label='TriAnd1',
            marker='^', color='#7570b3', **data_style)

ax.errorbar(triand2_c.l.degree, v_gsr(triand2_c).value, label='TriAnd2',
            marker='v', color='#43a2ca', **data_style)

ax.set_xticks([90, 120, 150, 180, 210, 240, 270])

ax.set_xlim(xlim)
ax.set_ylim(-160, 160)

ax.legend(loc='upper left', fontsize=14)

ax.set_xlabel(r'Galactic longitude, $l$ [${\rm deg}$]')
ax.set_ylabel(r'$v_{{\rm GSR}}$ [{0}]'.format((u.km/u.s).to_string(format='latex_inline')))

fig.tight_layout()
fig.set_facecolor('w')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6, 5))

for i in range(n_lines):
    idx = (gal_sim.l[:,i] > 80*u.deg) & (gal_sim.l[:,i] < 275*u.deg)
    d = gal_sim.distance.value[idx,i]
    ax.plot(gal_sim.l[idx,i].degree, d, lw=lws[i], **line_style)
    ax.text(270, d[-1]-4.25, 'R={0:.0f} kpc'.format(cyl.rho[0,i].value), 
            rotation=-25, **label_style)

ax.errorbar(gass_c.l.degree, gass_c.distance.value, 0.25 * gass_c.distance.value, label='GASS',
            marker='o', color='#1b9e77', **data_style)

ax.errorbar(a13_c.l.degree, a13_c.distance.value, 0.2 * a13_c.distance.value, label='A13',
            marker='s', color='#d95f02', **data_style)

ax.errorbar(triand1_c.l.degree, triand1_c.distance.value, 0.2 * triand1_c.distance.value, label='TriAnd1',
            marker='^', color='#7570b3', **data_style)

ax.errorbar(triand2_c.l.degree, triand2_c.distance.value, 0.2 * triand2_c.distance.value, label='TriAnd2',
            marker='v', color='#43a2ca', **data_style)

ax.set_xticks([90, 120, 150, 180, 210, 240, 270])

ax.set_xlim(xlim)
ax.set_ylim(5, 40)

ax.legend(loc='upper left', fontsize=14)

ax.set_xlabel(r'Galactic longitude, $l$ [${\rm deg}$]')
ax.set_ylabel(r'helio. distance, $d_\odot$ [${\rm kpc}$]')

fig.tight_layout()
fig.set_facecolor('w')

In [ ]:
fig = plt.figure()

_,bins,_ = plt.hist(triand1_c.distance.value[triand1_c.distance.value<40], 
                    bins='auto', alpha=0.5, label='TriAnd1')
_,bins,_ = plt.hist(triand2_c.distance.value, bins=bins, 
                    alpha=0.7, label='TriAnd2')
plt.legend(loc='upper right')
plt.xlabel('distance, $d$ [kpc]')
fig.set_facecolor('w')

## Simulate what stars on circular orbits in these longitudes would look like:

In [ ]:
n_per_line = 256

R = np.arange(15, 30+1, 5) * u.kpc
n_lines = len(R)
z = np.zeros(n_lines) * u.kpc
phi = np.linspace(0, 2*np.pi, n_per_line) * u.radian

cyl = coord.CylindricalRepresentation(rho=R[None], phi=phi[:,None], z=z[None])
v_cyl = coord.CylindricalDifferential(d_rho=np.zeros(cyl.shape)*u.km/u.s, 
                                      d_phi=(-220*u.km/u.s / cyl.rho).to(u.rad/u.Myr, u.dimensionless_angles()), 
                                      d_z=np.zeros(cyl.shape)*u.km/u.s)
cyl = cyl.with_differentials(v_cyl)
cart = cyl.represent_as(coord.CartesianRepresentation, coord.CartesianDifferential)

# disp = np.random.normal([0,0,0], [25,25,25], size=(n_sim, 3)).T * u.km/u.s
# cart = cart.with_differentials(cart.differentials['s'] + 
#                                coord.CartesianDifferential(disp))

gc_sim = coord.Galactocentric(cart, galcen_v_sun=coord.CartesianDifferential(-v_sun), z_sun=0*u.pc)
gal_sim = gc_sim.transform_to(coord.Galactic)

In [ ]:
data_style = dict(mew=0.5, mec='k', ms=5, ecolor='#999999', ls='none', alpha=0.8, zorder=10)
line_style = dict(color='#dddddd', ls='-', marker='', alpha=1)
lws = [2, 3, 4, 5]

label_style = dict(fontsize=13, va='center', color='#555555')
label_offset = [4, 4, 4, 8]

xlim = (275, 85)

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6, 5))

for i in range(n_lines):
    idx = (gal_sim.l[:,i] > 80*u.deg) & (gal_sim.l[:,i] < 275*u.deg)
    v = v_gsr(gal_sim[idx,i]).value
    ax.plot(gal_sim.l[idx,i].degree, v, lw=lws[i], **line_style)
    ax.text(270, v[-1]+label_offset[i], 'R={0:.0f} kpc'.format(cyl.rho[0,i].value), 
            rotation=10, **label_style)

ax.errorbar(gass_c.l.degree, v_gsr(gass_c).value, label='GASS',
            marker='o', color='#1b9e77', **data_style)

ax.errorbar(a13_c.l.degree, v_gsr(a13_c).value, label='A13',
            marker='s', color='#d95f02', **data_style)

ax.errorbar(triand_c.l.degree, v_gsr(triand_c).value, label='TriAnd',
            marker='^', color='#7570b3', **data_style)

ax.set_xticks([90, 120, 150, 180, 210, 240, 270])

ax.set_xlim(xlim)
ax.set_ylim(-160, 160)

ax.legend(loc='upper left', fontsize=14)

ax.set_xlabel(r'Galactic longitude, $l$ [${\rm deg}$]')
ax.set_ylabel(r'$v_{{\rm GSR}}$ [{0}]'.format((u.km/u.s).to_string(format='latex_inline')))

fig.tight_layout()
fig.savefig('../paper/figures/vgsr.pdf')

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6, 5))

for i in range(n_lines):
    idx = (gal_sim.l[:,i] > 80*u.deg) & (gal_sim.l[:,i] < 275*u.deg)
    d = gal_sim.distance.value[idx,i]
    ax.plot(gal_sim.l[idx,i].degree, d, lw=lws[i], **line_style)
    ax.text(270, d[-1]-4.25, 'R={0:.0f} kpc'.format(cyl.rho[0,i].value), 
            rotation=-25, **label_style)

ax.errorbar(gass_c.l.degree, gass_c.distance.value, 0.25 * gass_c.distance.value, label='GASS',
            marker='o', color='#1b9e77', **data_style)

ax.errorbar(a13_c.l.degree, a13_c.distance.value, 0.2 * a13_c.distance.value, label='A13',
            marker='s', color='#d95f02', **data_style)

ax.errorbar(triand_c.l.degree, triand_c.distance.value, 0.2 * triand_c.distance.value, label='TriAnd',
            marker='^', color='#7570b3', **data_style)

ax.set_xticks([90, 120, 150, 180, 210, 240, 270])

ax.set_xlim(xlim)
ax.set_ylim(5, 40)

ax.legend(loc='upper left', fontsize=14)

ax.set_xlabel(r'Galactic longitude, $l$ [${\rm deg}$]')
ax.set_ylabel(r'helio. distance, $d_\odot$ [${\rm kpc}$]')

fig.tight_layout()
fig.savefig('../paper/figures/dist.pdf')

In [ ]:
fig,axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
for i in range(n_lines):
    idx = (gal_sim.l[:,i] > 80*u.deg) & (gal_sim.l[:,i] < 275*u.deg)
    d = gal_sim.distance.value[idx,i]
    ax.plot(gal_sim.l[idx,i].degree, d, lw=lws[i], **line_style)
    ax.text(270, d[-1]-4.25, 'R={0:.0f} kpc'.format(cyl.rho[0,i].value), 
            rotation=-25, **label_style)

ax.errorbar(gass_c.l.degree, gass_c.distance.value, 0.25 * gass_c.distance.value, label='Mon/GASS',
            marker='o', color='#1b9e77', **data_style)

ax.errorbar(a13_c.l.degree, a13_c.distance.value, 0.2 * a13_c.distance.value, label='A13',
            marker='s', color='#d95f02', **data_style)

ax.errorbar(triand_c.l.degree, triand_c.distance.value, 0.2 * triand_c.distance.value, label='TriAnd',
            marker='^', color='#7570b3', **data_style)

ax.set_xticks([90, 120, 150, 180, 210, 240, 270])

ax.set_xlim(xlim)
ax.set_ylim(5, 40)

ax.legend(loc='upper left', fontsize=14)

ax.set_xlabel(r'Galactic longitude, $l$ [${\rm deg}$]')
ax.set_ylabel(r'helio. distance, $d_\odot$ [${\rm kpc}$]')

# ----------------------------------------------------

ax = axes[1]
for i in range(n_lines):
    idx = (gal_sim.l[:,i] > 80*u.deg) & (gal_sim.l[:,i] < 275*u.deg)
    v = v_gsr(gal_sim[idx,i]).value
    ax.plot(gal_sim.l[idx,i].degree, v, lw=lws[i], **line_style)
    ax.text(270, v[-1]+label_offset[i], 'R={0:.0f} kpc'.format(cyl.rho[0,i].value), 
            rotation=10, **label_style)

ax.errorbar(gass_c.l.degree, v_gsr(gass_c).value, label='Mon/GASS',
            marker='o', color='#1b9e77', **data_style)

ax.errorbar(a13_c.l.degree, v_gsr(a13_c).value, label='A13',
            marker='s', color='#d95f02', **data_style)

ax.errorbar(triand_c.l.degree, v_gsr(triand_c).value, label='TriAnd',
            marker='^', color='#7570b3', **data_style)

ax.set_xticks([90, 120, 150, 180, 210, 240, 270])

ax.set_xlim(xlim)
ax.set_ylim(-160, 160)

ax.set_xlabel(r'Galactic longitude, $l$ [${\rm deg}$]')
ax.set_ylabel(r'$v_{{\rm GSR}}$ [{0}]'.format((u.km/u.s).to_string(format='latex_inline')))

fig.tight_layout()

fig.savefig('../paper/figures/d_vgsr.pdf')

In [ ]:
a13_gc = a13_c.transform_to(gc_frame)
gass_gc = gass_c.transform_to(gc_frame)
triand_gc = triand_c.transform_to(gc_frame)

a13_cyl = a13_gc[a13_c.distance.value < 99.].represent_as('cylindrical')
gass_cyl = gass_gc.represent_as('cylindrical')
triand_cyl = triand_gc[triand_c.distance.value < 99].represent_as('cylindrical')

In [ ]:
np.repeat(a13_c.l.value[None], n_mc, axis=0).shape

In [ ]:
n_mc = 2048

def mc_xy_err(c, frac_d_err=0.2, n_mc=2048):
    d = c.distance.value[:,None]
    d_err = frac_d_err * d
    _gal = coord.Galactic(l=np.repeat(c.l.value[None], n_mc, axis=0) * u.degree, 
                          b=np.repeat(c.b.value[None], n_mc, axis=0) * u.degree, 
                          distance=np.random.normal(d, d_err, size=(d.size, 2048)).T * c.distance.unit)
    _gc = _gal.transform_to(gc_frame)
    xerr = np.std(_gc.x, axis=0).value
    yerr = np.std(_gc.y, axis=0).value
    zerr = np.std(_gc.z, axis=0).value
    Rerr = np.std(_gc.represent_as(coord.CylindricalRepresentation).rho, axis=0).value
    return xerr, yerr, zerr, Rerr

In [ ]:
fig,axes = plt.subplots(1, 2, figsize=(8, 5))

ax = axes[0]

shifts = [-2.5, -5, -7.5, -10]
offsets = [1.5, 2., 2.5, 4]
for i in range(n_lines):
    ax.plot(gc_sim.x.value[:, i], gc_sim.y.value[:, i], lw=lws[i], **line_style)
    d = gc_sim.y[:,i].min().value
    ax.text(-10+shifts[i], d+offsets[i], 'R={0:.0f} kpc'.format(cyl.rho[0,i].value), 
            rotation=-25, **label_style)

xerr, yerr, zerr, Rerr = mc_xy_err(gass_c)
axes[0].errorbar(gass_gc.x.value, gass_gc.y.value, label='GASS',
                 xerr=xerr, yerr=yerr,
                 marker='o', color='#1b9e77', **data_style)
axes[1].errorbar(np.sqrt(gass_gc.x.value**2 + gass_gc.y.value**2), gass_gc.z.value, label='Mon/GASS',
                 xerr=Rerr, yerr=zerr,
                 marker='o', color='#1b9e77', **data_style)

xerr, yerr, zerr, Rerr = mc_xy_err(a13_c)
axes[0].errorbar(a13_gc.x.value, a13_gc.y.value, label='A13',
                 xerr=xerr, yerr=yerr,
                 marker='s', color='#d95f02', **data_style)
axes[1].errorbar(np.sqrt(a13_gc.x.value**2 + a13_gc.y.value**2), a13_gc.z.value, label='A13',
                 xerr=Rerr, yerr=zerr,
                 marker='s', color='#d95f02', **data_style)

xerr, yerr, zerr, Rerr = mc_xy_err(triand_c)
axes[0].errorbar(triand_gc.x.value, triand_gc.y.value, label='TriAnd',
                 xerr=xerr, yerr=yerr,
                 marker='^', color='#7570b3', **data_style)
axes[1].errorbar(np.sqrt(triand_gc.x.value**2 + triand_gc.y.value**2), triand_gc.z.value, label='TriAnd',
                 xerr=Rerr, yerr=zerr,
                 marker='^', color='#7570b3', **data_style)

axes[1].legend(loc='upper left', fontsize=11)

axes[0].set_xlabel(r'$x$ [${\rm kpc}$]')
axes[0].set_ylabel(r'$y$ [${\rm kpc}$]')

axes[1].set_xlabel(r'$R$ [${\rm kpc}$]')
axes[1].set_ylabel(r'$z$ [${\rm kpc}$]')

axes[0].set_xticks(np.arange(-40, 0+1, 10))
axes[1].set_xticks(np.arange(10, 35+1, 5))

axes[0].text(-8.3, 0, r'$\odot$', va='center', ha='center', fontsize=18)
axes[1].text(8.3, 0, r'$\odot$', va='center', ha='center', fontsize=18)

for phi in [140, 180, 220]*u.deg:
    axes[0].plot([0, 50*np.cos(phi)], [0, 50*np.sin(phi)], 
                 marker='None', linestyle='--', zorder=-100, color='#aaaaaa')
    
axes[0].text(-37, 24, r'$\phi=140^\circ$', rotation=-40, **label_style)
axes[0].text(-39, 1.5, r'$\phi=180^\circ$', **label_style)
axes[0].text(-37, -24, r'$\phi=220^\circ$', rotation=40, **label_style)
    
axes[0].set_xlim(-40, 0)
axes[0].set_ylim(-30, 30)
axes[0].set_aspect('equal')

axes[1].set_xlim(6, 35.6)
axes[1].set_ylim(-20, 20)
axes[1].set_aspect('equal')

fig.tight_layout()
fig.savefig('../paper/figures/xy_Rz.pdf')

In [ ]:
data_style2 = dict(alpha=0.8, linewidth=0.5, edgecolor='k', vmin=120, vmax=210, zorder=100)

In [ ]:
fig,axes = plt.subplots(1, 2, figsize=(8, 5))

ax = axes[0]

shifts = [-2.5, -5, -7.5, -10]
offsets = [1.5, 2., 2.5, 4]
for i in range(n_lines):
    ax.plot(gc_sim.x.value[:, i], gc_sim.y.value[:, i], lw=lws[i], **line_style)
    d = gc_sim.y[:,i].min().value
    ax.text(-10+shifts[i], d+offsets[i], 'R={0:.0f} kpc'.format(cyl.rho[0,i].value), 
            rotation=-25, **label_style)

xerr, yerr, zerr, Rerr = mc_xy_err(gass_c)
axes[0].errorbar(gass_gc.x.value, gass_gc.y.value, 
                 xerr=xerr, yerr=yerr,
                 marker=None, color='#1b9e77', **data_style)
axes[1].errorbar(np.sqrt(gass_gc.x.value**2 + gass_gc.y.value**2), gass_gc.z.value,
                 xerr=Rerr, yerr=zerr,
                 marker=None, color='#1b9e77', **data_style)

R = np.sqrt(gass_gc.x.value**2 + gass_gc.y.value**2)
phi = np.degrees(np.arctan2(gass_gc.y.value, gass_gc.x.value)) % 360
axes[0].scatter(gass_gc.x.value, gass_gc.y.value, c=phi, **data_style2)
axes[1].scatter(R, gass_gc.z.value, c=phi, **data_style2)

xerr, yerr, zerr, Rerr = mc_xy_err(a13_c)
axes[0].errorbar(a13_gc.x.value, a13_gc.y.value, label='A13',
                 xerr=xerr, yerr=yerr,
                 marker=None, color='#d95f02', **data_style)
axes[1].errorbar(np.sqrt(a13_gc.x.value**2 + a13_gc.y.value**2), a13_gc.z.value,
                 xerr=Rerr, yerr=zerr,
                 marker=None, color='#d95f02', **data_style)

R = np.sqrt(a13_gc.x.value**2 + a13_gc.y.value**2)
phi = np.degrees(np.arctan2(a13_gc.y.value, a13_gc.x.value)) % 360
axes[0].scatter(a13_gc.x.value, a13_gc.y.value, c=phi, **data_style2)
axes[1].scatter(R, a13_gc.z.value, c=phi, **data_style2)

xerr, yerr, zerr, Rerr = mc_xy_err(triand_c)
axes[0].errorbar(triand_gc.x.value, triand_gc.y.value, label='TriAnd',
                 xerr=xerr, yerr=yerr,
                 marker=None, color='#7570b3', **data_style)
axes[1].errorbar(np.sqrt(triand_gc.x.value**2 + triand_gc.y.value**2), triand_gc.z.value,
                 xerr=Rerr, yerr=zerr,
                 marker=None, color='#7570b3', **data_style)

R = np.sqrt(triand_gc.x.value**2 + triand_gc.y.value**2)
phi = np.degrees(np.arctan2(triand_gc.y.value, triand_gc.x.value)) % 360
axes[0].scatter(triand_gc.x.value, triand_gc.y.value, c=phi, **data_style2)
axes[1].scatter(R, triand_gc.z.value, c=phi, **data_style2)

axes[0].legend(loc='lower left', fontsize=12)

axes[0].set_xlabel(r'$x_\odot$ [${\rm kpc}$]')
axes[0].set_ylabel(r'$y_\odot$ [${\rm kpc}$]')

axes[1].set_xlabel(r'$R$ [${\rm kpc}$]')
axes[1].set_ylabel(r'$z$ [${\rm kpc}$]')

axes[0].set_xticks(np.arange(-40, 0+1, 10))
axes[1].set_xticks(np.arange(10, 35+1, 5))

axes[0].text(-8.3, 0, r'$\odot$', va='center', ha='center', fontsize=18)
# axes[1].text(8.3, 0, r'$\odot$', va='center', ha='center', fontsize=18)

axes[0].set_xlim(-40, 0)
axes[0].set_ylim(-30, 30)
axes[0].set_aspect('equal')

axes[1].set_xlim(9, 35.6)
axes[1].set_ylim(-20, 20)
axes[1].set_aspect('equal')

fig.tight_layout()

## TriAnd RR Lyrae

In [ ]:
all_rrl = np.genfromtxt("/Users/adrian/projects/triand-rrlyrae/data/apw_velocities.csv", 
                        skip_header=0, dtype=None, names=True, delimiter=',')
print(all_rrl.dtype.names)

rr_c = coord.ICRS(ra=all_rrl['ra']*u.deg, dec=all_rrl['dec']*u.deg,
                  radial_velocity=all_rrl['Vsys']*u.km/u.s)
rrl_l = rr_c.transform_to(coord.Galactic).l.degree
rrl_vgsr = v_gsr(rr_c).value
rrl_verr = all_rrl['Err']

# ix = np.ones(len(all_rrl)).astype(bool)
ix = (all_rrl['dist'] > 15) & (all_rrl['dist'] < 22.)
rrl = all_rrl[ix]

# add the CSS star
rrl_l = np.append(rrl_l[ix], 111.285)
rrl_vgsr = np.append(rrl_vgsr[ix], -127.9)
rrl_verr = np.append(rrl_verr[ix], 15.)
len(rrl_l)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,4.5))

ax.errorbar(triand1_c.l.degree, v_gsr(triand1_c).value, label='M giant',
            marker='^', color='#7570b3', **data_style)

ax.errorbar(rrl_l, rrl_vgsr, rrl_verr, linestyle='none', label='RR Lyrae', 
            marker='o', ms=4., color='#2c7fb8', mec='#333333', mew=0.5,
            ecolor='#666666', elinewidth=1, zorder=10)

ax.set_xlabel(r"$l\,{\rm [deg]}$", fontsize=26)
ax.set_ylabel(r"$v_{\rm GSR}\,[{\rm km} \, {\rm s}^{-1}]$")

ax.set_xlim(165, 95)
ax.set_ylim(-200, 200)

ax.set_title('TriAnd')

ax.legend(loc='upper left', fontsize=14)

fig.tight_layout()

fig.savefig('../paper/figures/triand_rrlyrae.pdf')

## Cartoon

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline

_x = [4, 8, 15, 30, 40]
_y = [0.025, 0.05, 4, 11, 15]
plt.plot(_x, _y)

_amplitude = InterpolatedUnivariateSpline(_x, _y, k=1, ext=1)

In [ ]:
def wavelength(R):
    x1,y1 = [8, 8.]
    x2,y2 = [35, 14]
    return (abs(R) - x1)*(y2-y1)/(x2-x1) + y1

def amplitude(R):
    return - _amplitude(R)

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6,6))

style = dict(linewidth=4, alpha=0.7, marker='None', color='#555555')

x = np.linspace(3, 35, 1024)
x0 = 8

ax.plot(x + 2.7, amplitude(x) * np.sin(2*np.pi*(x - x0)/wavelength(x)), 
        linestyle='-', label=r"$\phi=140^\circ$", **style)
ax.plot(x, amplitude(x) * np.sin(2*np.pi*(x - x0)/wavelength(x)), 
        linestyle='--', label=r"$\phi=180^\circ$", **style)
ax.plot(x - 2.7, amplitude(x) * np.sin(2*np.pi*(x - x0)/wavelength(x)), 
        linestyle=':', label=r"$\phi=220^\circ$", **style)

ax.set_ylim(-12, 12)
ax.set_xlim(6, 32)

ax.set_xlabel('$R$ [kpc]')
ax.set_ylabel('$z$ [kpc]')

# ax.text(x0, 0, r'$\mathbf{\odot}$', va='center', ha='center', fontsize=18)
ax.plot(x0, 0, marker='.', ms=8)
ax.plot(x0, 0, marker='o', mfc='none', mec='k', mew=2., ms=16)

ax.text(10.75, -3, 'Mon/GASS', ha='center', fontsize=18)
ax.text(15.5, 5, 'Mon/GASS+A13', ha='center', fontsize=18)
ax.text(22.5, -8.9, 'TriAnd', ha='center', fontsize=18)

ax.legend(loc='lower left', fontsize=16)

fig.tight_layout()
ax.set_aspect('equal')

fig.savefig('../paper/figures/cartoon.pdf')

Mon/GASS at 140-180, A13 at ~180, TriAnd more at ~220